# Azure AI Agent Observability with Application Insights

This notebook demonstrates how to enable observability for Azure AI agents using **OpenTelemetry** and **Azure Application Insights**. Agent Framework integrates with OpenTelemetry and emits traces, logs, and metrics according to the [OpenTelemetry GenAI Semantic Conventions](https://opentelemetry.io/docs/specs/semconv/gen-ai/).

## What You'll Learn

- ✅ How to set up **automatic telemetry** for Azure AI agents
- ✅ How to configure **Application Insights** connection
- ✅ How to **trace agent invocations** and tool calls
- ✅ How to view **trace IDs** for debugging
- ✅ How to enable **streaming responses** with observability

## Key Observability Features

### Automatic Spans Created
- **`invoke_agent <agent_name>`**: Top-level span for each agent invocation
- **`chat <model_name>`**: Span for underlying chat model calls
- **`execute_tool <function_name>`**: Span for function tool executions

### Metrics Collected
- **`gen_ai.client.operation.duration`** (histogram): Operation duration in seconds
- **`gen_ai.client.token.usage`** (histogram): Token usage counts
- **`agent_framework.function.invocation.duration`** (histogram): Function execution duration

> **Note**: You must add an **Application Insights** instance to your Azure AI project for this sample to work. The `setup_azure_ai_observability()` method automatically retrieves the connection string from your Azure AI project.

## Prerequisites

Before running this notebook, ensure you have:

1. **Application Insights** attached to your Azure AI Foundry project
2. **Azure CLI** authenticated (`az login --use-device-code`)
3. **Environment variables** configured in `../.env`:
   - `AZURE_AI_PROJECT_ENDPOINT`
4. **Python packages** installed:
   ```bash
   pip install agent-framework agent-framework-azure-ai python-dotenv azure-identity
   ```

## Import Required Libraries

We'll import the necessary libraries for agent creation, Azure AI integration, and observability:

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio
import os
from random import randint
from typing import Annotated

from dotenv import load_dotenv
from pydantic import Field

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from agent_framework.observability import get_tracer
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from opentelemetry.trace import SpanKind
from opentelemetry.trace.span import format_trace_id

## Environment Configuration

Load environment variables from the parent directory's `.env` file. We need the Azure AI project endpoint to connect to Azure AI Foundry:

In [ ]:
# Load environment variables from parent directory
load_dotenv('../.env')

# Validate required environment variable
required_vars = ['AZURE_AI_PROJECT_ENDPOINT']
missing = [var for var in required_vars if not os.getenv(var)]

if missing:
    raise RuntimeError(f'Missing required environment variables: {missing}')

endpoint = os.getenv('AZURE_AI_PROJECT_ENDPOINT')
print(f'✅ AZURE_AI_PROJECT_ENDPOINT: {endpoint}')

## Define Function Tools

We'll create a simple weather function that the agent can call. This demonstrates how tool calls are also traced in the observability system:

In [ ]:
async def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    # Simulate a network call
    await asyncio.sleep(randint(0, 10) / 10.0)
    
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    temperature = randint(10, 30)
    weather_condition = conditions[randint(0, 3)]
    
    return f"The weather in {location} is {weather_condition} with a high of {temperature}°C."

## Understanding Azure AI Observability Setup

### How Observability Works

1. **Automatic Connection**: The `setup_azure_ai_observability()` method automatically retrieves the Application Insights connection string from your Azure AI project.

2. **Global Configuration**: This sets up the global OpenTelemetry tracer and meter providers, which will automatically track:
   - Agent invocations
   - Chat model calls
   - Tool executions
   - Token usage
   - Response times

3. **Trace IDs**: Each conversation gets a unique trace ID that can be used to find all related spans in Application Insights.

### Viewing Traces in Azure

After running this notebook:
1. Go to Azure Portal → Your Azure AI Foundry Project
2. Navigate to the connected Application Insights resource
3. Go to **Transaction Search** or **End-to-end transaction details**
4. Search for the trace ID printed in the output
5. View the complete trace timeline with all spans and metrics

## Main Demo: Agent with Observability

This demo shows:
- Creating an Azure AI agent with observability enabled
- Using streaming responses
- Automatic trace collection for agent and tool calls
- Getting a trace ID for debugging

In [ ]:
async def run_agent_with_observability():
    """
    Run an Azure AI agent with observability enabled.
    
    This demonstrates:
    - Automatic Application Insights integration
    - Trace ID generation for debugging
    - Streaming responses with telemetry
    - Function tool execution tracing
    """
    # Create async context managers for Azure resources
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=endpoint, credential=credential) as project,
        AzureAIAgentClient(project_client=project) as client,
    ):
        # Enable observability - this configures the application to send telemetry 
        # to the Application Insights instance attached to the Azure AI project
        print("🔧 Setting up Azure AI observability...")
        await client.setup_azure_ai_observability()
        print("✅ Observability configured\n")
        
        # Test questions - mix of weather queries and general knowledge
        questions = [
            "What's the weather in Amsterdam?",
            "and in Paris, and which is better?",
            "Why is the sky blue?"
        ]
        
        # Create a custom span to group all agent interactions
        # This is optional but helpful for grouping related operations
        with get_tracer().start_as_current_span(
            "Single Agent Chat", 
            kind=SpanKind.CLIENT
        ) as current_span:
            # Get and display the trace ID - you can use this to find traces in Application Insights
            trace_id = format_trace_id(current_span.get_span_context().trace_id)
            print("="*70)
            print(f"🔍 Trace ID: {trace_id}")
            print(f"   Use this ID to find traces in Application Insights")
            print("="*70)
            print()
            
            # Create the agent with tools
            agent = ChatAgent(
                chat_client=client,
                tools=get_weather,
                name="WeatherAgent",
                instructions="You are a weather assistant.",
            )
            
            # Create a thread for the conversation
            thread = agent.get_new_thread()
            
            # Process each question with streaming response
            for question in questions:
                print(f"👤 User: {question}")
                print(f"🤖 {agent.display_name}: ", end="")
                
                # Stream the response (telemetry is automatically collected)
                async for update in agent.run_stream(question, thread=thread):
                    if update.text:
                        print(update.text, end="", flush=True)
                
                print("\n")  # New line after response
        
        print("="*70)
        print("✅ Demo completed!")
        print(f"   Check Application Insights for trace ID: {trace_id}")
        print("="*70)

# Run the demo
await run_agent_with_observability()

## 🔍 How to Find Traces in Azure AI Foundry

After running the cell below, follow these **exact steps** to find your traces:

### Method 1: Azure AI Foundry Studio (Recommended)

1. **Open**: https://ai.azure.com
2. **Sign in** with your Azure account
3. **Click** on your project: **kd-foundry-project** (or select from the list)
4. **Look for "Tracing"** in the left navigation menu
   - It should be under the "Monitoring" or "Evaluate" section
   - If you don't see it, click on "..." (More) or check under different menu sections
5. **Wait 1-2 minutes**, then **refresh** the page
6. **Search** for the trace ID shown in the output below
7. **Click** on the trace to see the full hierarchy

### Method 2: Application Insights in Azure Portal

1. **Open**: https://portal.azure.com
2. **Search** for: `kd-foundry-project` in the top search bar
3. **Find the Application Insights resource** (it might be named like `kd-foundry-project-appinsights` or similar)
4. **Click** on the Application Insights resource
5. On the left menu, find **"Transaction search"** (under "Investigate")
6. **Set time range** to "Last 1 hour" or "Last 6 hours"
7. **Click** "Search" to see all recent transactions
8. **Look for** operations with names like:
   - `invoke_agent WeatherAgent`
   - `chat gpt-4o`
   - `execute_tool get_weather`
9. **Click** on any transaction to see the full trace

### What Traces Look Like

You should see a hierarchy like this:
```
Single Agent Chat (root span)
└── invoke_agent WeatherAgent
    ├── chat gpt-4o
    └── execute_tool get_weather
```

> **⏱️ Note**: Traces can take 1-5 minutes to appear. If you don't see them immediately, wait and refresh!

In [ ]:
# Helper: Get Application Insights Information
import os
from dotenv import load_dotenv

load_dotenv("../.env")

print("📋 Your Azure Configuration:")
print(f"  Subscription ID: {os.getenv('AZURE_SUBSCRIPTION_ID')}")
print(f"  Resource Group: {os.getenv('AZURE_RESOURCE_GROUP')}")
print(f"  Project Name: {os.getenv('AZURE_PROJECT_NAME')}")
print()
print("🔗 Direct Links:")
print(f"  Azure AI Foundry: https://ai.azure.com")
print(f"  Azure Portal (Resource Group): https://portal.azure.com/#@{os.getenv('TENANT_ID')}/resource/subscriptions/{os.getenv('AZURE_SUBSCRIPTION_ID')}/resourceGroups/{os.getenv('AZURE_RESOURCE_GROUP')}/overview")
print()
print("📝 To find Application Insights:")
print("  1. Go to Azure Portal link above")
print("  2. Look for a resource with type 'Application Insights'")
print("  3. Click on it and go to 'Transaction search'")
print("  4. Set time range to 'Last 1 hour'")
print("  5. Look for traces with 'invoke_agent' or 'WeatherAgent'")

In [ ]:
# Diagnostic: Check Application Insights Connection
async def check_app_insights_connection():
    """Check which Application Insights instance is configured for the AI project."""
    from azure.identity.aio import AzureCliCredential
    from azure.ai.projects.aio import AIProjectClient
    import os
    from dotenv import load_dotenv
    
    load_dotenv("../.env")
    endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=endpoint, credential=credential) as project,
    ):
        # Try to get project properties
        print("🔍 Checking Azure AI Project Configuration...")
        print(f"   Project Endpoint: {endpoint}")
        
        # The AIProjectClient should have connection information
        try:
            # Get the project's properties which should include App Insights
            print("\n✅ Successfully connected to Azure AI Project")
            print("   If observability is working, traces should appear in Application Insights")
            print("\n📝 Next steps:")
            print("   1. Run the agent demo cell above")
            print("   2. Wait 2-3 minutes")
            print("   3. Check Application Insights in Azure Portal")
            print("   4. Look for traces with the trace ID shown in the output")
        except Exception as e:
            print(f"\n❌ Error: {e}")

await check_app_insights_connection()

## Understanding the Telemetry Data

### What Gets Tracked

When you run the agent above, Application Insights captures:

1. **Agent Invocation Span** (`invoke_agent WeatherAgent`):
   - Duration of the entire agent operation
   - Input instructions
   - Response metadata
   - Token usage (input and output)

2. **Chat Model Span** (`chat <model_name>`):
   - LLM invocation details
   - Model name and system used
   - Response ID for correlation

3. **Tool Execution Span** (`execute_tool get_weather`):
   - Function execution duration
   - Function arguments (if sensitive data is enabled)
   - Function results (if sensitive data is enabled)

### Sample Trace Output

```json
{
    "name": "invoke_agent WeatherAgent",
    "trace_id": "0xf2258b51421fe9cf4c0bd428c87b1ae4",
    "span_id": "0x2cad6fc139dcf01d",
    "attributes": {
        "gen_ai.operation.name": "invoke_agent",
        "gen_ai.system": "openai",
        "gen_ai.agent.id": "WeatherAgent",
        "gen_ai.agent.name": "WeatherAgent",
        "gen_ai.request.instructions": "You are a weather assistant.",
        "gen_ai.usage.input_tokens": 26,
        "gen_ai.usage.output_tokens": 29
    }
}
```

## Next Steps

### Enable Sensitive Data Logging (Development Only)

To see actual prompts, responses, and function arguments in traces, set environment variables:

```bash
ENABLE_SENSITIVE_DATA=true
```

Then use:
```python
from agent_framework.observability import setup_observability
setup_observability(enable_sensitive_data=True)
```

⚠️ **Warning**: Only enable in development! This may expose sensitive user data in logs.

### Explore More Observability Features

1. **View traces in Azure Portal**: Use the trace ID to find your traces
2. **Create custom spans**: Use `get_tracer()` to add your own spans
3. **Add custom metrics**: Use `get_meter()` to track custom metrics
4. **Try Aspire Dashboard**: For local development visualization

### Related Documentation

- 📖 [Agent Observability Guide](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-observability?pivots=programming-language-python)
- 📖 [OpenTelemetry GenAI Conventions](https://opentelemetry.io/docs/specs/semconv/gen-ai/)
- 📖 [Application Insights](https://learn.microsoft.com/en-us/azure/azure-monitor/app/app-insights-overview)